In [22]:
from sklearn.model_selection import train_test_split
from sklearn.discriminant_analysis \
import LinearDiscriminantAnalysis
from sklearn.discriminant_analysis \
import QuadraticDiscriminantAnalysis
import numpy as np
import matplotlib.pyplot as plt
import scipy.linalg as LA
import matplotlib.patches as mpatches
from matplotlib import colors
from sklearn.model_selection import train_test_split
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import sklearn.neighbors as neighbors
from matplotlib.colors import ListedColormap
from sklearn import datasets, neighbors
from scipy.stats import multivariate_normal
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import ConfusionMatrixDisplay

# --------------------------------------------------------------------
# 生成兩群組模擬資料
def param(n1, n2, mean1, mean2, mean3, mean4, val1, val2):
    m1, m2 = np.array([mean1, mean2]), np.array([mean3, mean4])
    Cov1 = np.array([[1, val1], [val1, 1]])
    Cov2 = np.array([[1, val2], [val2, 1]])
    mvn1 = multivariate_normal(mean = m1, cov = Cov1)
    mvn2 = multivariate_normal(mean = m2, cov = Cov2)
    A = mvn1.rvs(n1)
    B = mvn2.rvs(n2)
    Xvar = np.vstack((A, B))#2000*1矩陣\
    y = np.hstack((np.zeros(n1), np.ones(n2)))#2000
    param = np.c_[Xvar, y]
    return param

# -------------------------------------------------------------------
# 生成三群組模擬資料
def param3(n1, n2, n3, mean1, mean2, mean3, mean4, mean5, mean6, val1, val2, val3):
    m1, m2, m3 = np.array([mean1, mean2]), np.array([mean3, mean4]), np.array([mean5, mean6])
    Cov1 = np.array([[1, val1], [val1, 1]])
    Cov2 = np.array([[1, val2], [val2, 1]])
    Cov3 = np.array([[1, val3], [val3, 1]])
    mvn1 = multivariate_normal(mean = m1, cov = Cov1)
    mvn2 = multivariate_normal(mean = m2, cov = Cov2)
    mvn3 = multivariate_normal(mean = m3, cov = Cov3)
    A = mvn1.rvs(n1)
    B = mvn2.rvs(n2)
    C = mvn2.rvs(n3)
    Xvar = np.vstack((A, B, C))#2000*1矩陣\
    y = np.hstack((np.zeros(n1), np.ones(n2), np.ones(n3)+1))#2000
    param3 = np.c_[Xvar, y]
    return param3
    

In [25]:
# ---------------------------------------------------------------
# 兩群組資料生成
# ------------------------------------------------
D = param(500, 500, 2, 2, 4, 1, 0.9, 0.3)
data = X = D[:, 0:2]
label = y = D[:, 2]
# --------------------------------------------------------------
# 加廣線性迴歸與簡單線性迴歸
from sklearn.linear_model import LinearRegression

# -----------------------------------------------
n =100 #模擬次數
Lineartrainerror = np.zeros(n)
Lineartesterror = np.zeros(n)
Augmentedtrainerror = np.zeros(n)
Augmentedtesterror = np.zeros(n)
Logistictrainerror = np.zeros(n)
Logistictesterror = np.zeros(n)

LDAtrainingError = np.zeros(n) #用來存取訓練誤差
LDAtestingError = np.zeros(n) #存取測試誤差
QDAtrainingError = np.zeros(n)
QDAtestingError = np.zeros(n)
KNNtrainingError = np.zeros(n)
KNNtraingError = np.zeros(n)

ANNtrainerror = np.zeros(n)
ANNtesterror = np.zeros(n)

# ------------------------------------------------
#載入套件
Lda = LinearDiscriminantAnalysis(tol = 1e-6)
Qda = QuadraticDiscriminantAnalysis(tol = 1e-6, store_covariance = True)
Mdl = LinearRegression()
from sklearn.linear_model import LogisticRegression
logreg = LogisticRegression(multi_class='multinomial',solver='lbfgs',class_weight='balanced',max_iter=1000)

K1 = 5
weights = "uniform"
Knn = neighbors.KNeighborsClassifier(K1, weights = weights)

hidden_layers = (30, )
solver = "adam"
clf = MLPClassifier(max_iter = 10000, solver = solver,
hidden_layer_sizes = hidden_layers, verbose = True,
activation = "logistic", tol = 1e-6, random_state = 0)

# ------------------------------------------------
for i in range(n) :
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    # --------------------------------------------------
    Mdl.fit(X_train, y_train) 
    y_hat = Mdl.predict(X_train) 
    y_pretrain = [1 if i > 0.5 else 0 for i in y_hat]
    Lineartrainerror[i] = 1- np.mean(y_pretrain == y_train)

    y_hat = Mdl.predict(X_test) 
    y_pretest = [1 if i > 0.5 else 0 for i in y_hat]
    Lineartesterror[i] = 1 - np.mean(y_pretest == y_test)
    # ---------------------------------------------------
    x1 = X_train[:,0:1] 
    x2 = X_train[:,1:2]
    XX = np.hstack((x1, x2, x1 * x2, x1 ** 2, x2 ** 2))
    Mdl.fit(XX, y_train) 
    y_hat = Mdl.predict(XX) 
    y_pretrain = [1 if i > 0.5 else 0 for i in y_hat]
    Augmentedtrainerror[i] = 1 - np.mean(y_pretrain == y_train)

    x1 = X_test[:,0:1]
    x2 = X_test[:,1:2]
    XX = np.hstack((x1, x2, x1 * x2, x1 ** 2, x2 ** 2))
    y_hat = Mdl.predict(XX) 
    y_pretest = [1 if i > 0.5 else 0 for i in y_hat]
    Augmentedtesterror[i] = 1 - np.mean(y_pretest == y_test)
    # ---------------------------------------------------
    logreg.fit(X_train, y_train)
    y_predict = logreg.predict(X_test)
    Logistictrainerror[i] = 1 - logreg.score(X_train, y_train)
    Logistictesterror[i] = 1 - logreg.score(X_test, y_test)
    # ---------------------------------------------------
    Lda.fit(X_train, y_train)
    Lda.predict(X_test)
    LDAtrainingError[i] = 1 - Lda.score(X_train, y_train)
    LDAtestingError[i] = 1 - Lda.score(X_test, y_test)
    # ---------------------------------------------------
    Qda.fit(X_train, y_train)
    Qda.predict(X_test)
    QDAtrainingError = 1 - Qda.score(X_train, y_train)
    QDAtestingError = 1 - Qda.score(X_test, y_test)
    # ---------------------------------------------------
    Knn.fit(X_train, y_train)
    Knn.predict(X_test)
    KNNtrainingError1 = 1 - Knn.score(X_train, y_train)
    KNNtestingError1 = 1 - Knn.score(X_test, y_test)
    # ---------------------------------------------------
    #clf.fit(X_train, y_train)
    #y_test_hat = clf.predict(X_test)
    #ANNtrainerror[i] = 1 - clf.score(X_train, y_train)
    #ANNtesterror[i] = 1 - clf.score(X_test, y_test)

#print出bootstrapping抽取K次的訓練誤差平均值
print("Linear training Error:{:.4f}".format(Lineartrainerror.mean()))
print("Linear testing Error:{:.4f}".format(Lineartesterror.mean()))
print("Aug training Error:{:.4f}".format(Augmentedtrainerror.mean()))
print("Aug testing Error:{:.4f}".format(Augmentedtesterror.mean()))

print("LDA training Error:{:.4f}".format(LDAtrainingError.mean()))#除了平均數也可以看看變異數
print("LDA testing Error:{:.4f}".format(LDAtestingError.mean()))
print("QDA training Error:{:.4f}".format(QDAtrainingError.mean()))#除了平均數也可以看看變異數
print("QDA testing Error:{:.4f}".format(QDAtestingError.mean()))
print("KNN training Error1:{:.4f}".format(KNNtrainingError1.mean()))#除了平均數也可以看看變異數
print("KNN testing Error1:{:.4f}".format(KNNtestingError1.mean()))

print("ANN training Error1:{:.4f}".format(ANNtrainerror.mean()))#除了平均數也可以看看變異數
print("ANN testing Error1:{:.4f}".format(ANNtesterror.mean()))



Linear training Error:0.0541
Linear testing Error:0.0552
Aug training Error:0.0309
Aug testing Error:0.0297
LDA training Error:0.0541
LDA testing Error:0.0552
QDA training Error:0.0300
QDA testing Error:0.0350
KNN training Error1:0.0312
KNN testing Error1:0.0350
ANN training Error1:0.0000
ANN testing Error1:0.0000


In [26]:
print(Lineartrainerror.mean())
print(Lineartesterror.mean())
print(Augmentedtrainerror.mean())
print(Augmentedtesterror.mean())
print(Logistictrainerror.mean())
print(Logistictesterror.mean())

print(LDAtrainingError.mean())
print(LDAtestingError.mean())
print(QDAtrainingError.mean())
print(QDAtestingError.mean())
print(KNNtrainingError1.mean())
print(KNNtestingError1.mean())
#print(ANNtrainerror.mean())
#print(ANNtesterror.mean())


0.054075000000000005
0.05520000000000001
0.03088749999999999
0.02970000000000003
0.031187499999999993
0.02925000000000003
0.054075000000000005
0.05520000000000001
0.030000000000000027
0.03500000000000003
0.03125
0.03500000000000003


In [19]:
D = param(500, 500, 2, 2, 4, 1, 0.9, 0.3)
data = X = D[:, 0:2]
label = y = D[:, 2]
# --------------------------------------------------------------
# 加廣線性迴歸與簡單線性迴歸
from sklearn.linear_model import LinearRegression

# -----------------------------------------------
n =100 #模擬次數
Lineartrainerror = np.zeros(n)
Lineartesterror = np.zeros(n)
Augmentedtrainerror = np.zeros(n)
Augmentedtesterror = np.zeros(n)

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2)
    # --------------------------------------------------
Mdl.fit(X_train, y_train) 
y_hat = Mdl.predict(X_train) 
y_pretrain = [1 if i > 0.5 else 0 for i in y_hat]
print("Accuracy in training for augmented regression: {:.4f}".format(1 - np.mean(y_pretrain == y_train)))
Lineartrainerror = 1 - np.mean(y_pretrain == y_train)

y_hat = Mdl.predict(X_test) 
y_pretest = [1 if i > 0.5 else 0 for i in y_hat]
print("Accuracy in training for augmented regression: {:.4f}".format(1 - np.mean(y_pretest == y_test)))
#Lineartesterror[i] = np.mean(y_pretest == y_test)

Accuracy in training for augmented regression: 0.0563
Accuracy in training for augmented regression: 0.0400
